In [ ]:
import os
PROJ_DIR = os.path.expanduser("~/projects/bayesian_symbolic_physics")

import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use("bmh")

In [ ]:
def reorder(people, oracle, order):
    As = []
    for i in range(10):
        opt = np.array(oracle[i])[order]
        for j in range(6):
            idx = j + 6 * i
            A = []
            for k in range(3):
                a = people[idx,:,opt[k]]
                A.append(a)
            A = np.stack(A).transpose()
            As.append(A)
    A = np.stack(As)
    return A

def plot_matrix(mat, labels, rotate=False):
    fig, ax = plt.subplots(figsize=(2.5, 2.5))
    im = ax.imshow(mat, vmin=0, vmax=1)

    # We want to show all ticks...
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
    # ... and remove grids
    ax.grid(False)

    if rotate:
        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(labels)):
        for j in range(len(labels)):
            text = ax.text(j, i, f"{mat[i, j]:.3f}",
                           ha="center", va="center", color="w")

def compute_acc(mat):
    acc = []
    for idx in range(60):
        for k in range(3):
            val = mat[idx,k,:]
            if val.sum() > 0:
                acc.append(val[k])
    return np.mean(acc)

def compute_mean(mat):
    matMean = np.zeros([3, 3])
    n = np.zeros(3)
    for idx in range(60):
        for k in range(3):
            val = mat[idx,k,:]
            if val.sum() > 0:
                matMean[k,:] += val
                n[k] += 1
    matMean = matMean / np.expand_dims(n, axis=1)
    return matMean

In [ ]:
# When opened you should get a matrix that goes stim x property X property. 
# The properties are in ascending order. 
# So, for example, peopleMass is a 60 x 3 x 3; 
# the [i,j,k] entry E there is a number between 0-1 that indicates that in stimuli i, when the ground truth was j, E% of people estimated it was property k. 

# ``For mass, the properties are light/medium/heavy (corresponding to 1, 3, 9)''
peopleMass     = pickle.load(open( f"{PROJ_DIR}/paper/raw/peopleMass.p",     "rb"), encoding="bytes")
# ``For friction ('goo'), the properties are smooth/rough/very rough (corresponding to 0, 5, 20)''
peopleGoo      = pickle.load(open( f"{PROJ_DIR}/paper/raw/peopleGoo.p",      "rb"), encoding="bytes")
# ``For global, it should be none/up/down/left/right''
peopleGlobal   = pickle.load(open( f"{PROJ_DIR}/paper/raw/peopleGlobal.p",   "rb"), encoding="bytes")
# For pairwise, it should be attract/repel/none
peoplePairwise = pickle.load(open( f"{PROJ_DIR}/paper/raw/peoplePairwise.p", "rb"), encoding="bytes")

In [ ]:
# Red, Yellow, Blue
oracleMass = [
    [1, 0, 2], [2, 1, 0], [0, 2, 1], [1, 0, 2], [2, 1, 0],
    [0, 2, 1], [0, 2, 1], [1, 0, 2], [2, 1, 0], [1, 0, 2],
]
# -> Red, Blue, Yellow
orderMass = [0, 2, 1]

peopleMassR = reorder(peopleMass, oracleMass, orderMass)

print(compute_acc(peopleMassR))

peopleMassMean = compute_mean(peopleMassR)
plot_matrix(peopleMassMean, ["Light", "Medium", "Heavy"])

In [ ]:
bspMassMean = np.array([[0.522727, 0.318182, 0.159091], 
                        [0.340909, 0.454545, 0.204545], 
                        [0.386364, 0.386364, 0.227273]])
plot_matrix(bspMassMean, ["Light", "Medium", "Heavy"])

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
ax.scatter(peopleMassMean, bspMassMean)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

In [ ]:
# Green, Purple, Brown
oracleGoo = [
    [2, 0, 1], [0, 1, 2], [1, 2, 0], [2, 0, 1], [2, 0, 1],
    [0, 1, 2], [0, 1, 2], [1, 2, 0], [1, 2, 0], [2, 0, 1],
]
# -> Brown, Green, Purple
orderGoo = [2, 0, 1]

peopleGooR = reorder(peopleGoo, oracleGoo, orderGoo)

print(compute_acc(peopleGooR))

plot_matrix(compute_mean(peopleGooR), ["Smooth", "Weak", "Strong"])

In [ ]:
oracleGlobal = [0, 0, 0, 0, 0, 0, 3, 4, 2, 1]
acc = []
for i in range(10):
    for j in range(6):
        idx = j + 6 * i
        acc.append(peopleGlobal[idx,oracleGlobal[i]])
np.mean(acc)

In [ ]:
bspMassMean = np.array([[0.295455, 0.181818, 0.522727], 
                        [0.363636, 0.227273, 0.409091], 
                        [0.386364, 0.227273, 0.386364]])
plot_matrix(bspMassMean, ["Smooth", "Weak", "Strong"])

In [ ]:
import cv2

vpath = "~/projects/bayesian_symbolic/data/ullman/world1_1.mp4"

frames = []
cap = cv2.VideoCapture(vpath)
while True:
    ret, frame = cap.read()
    frames.append(frame)
    if not ret: break
        
def mix(ws, frames):
    retval = []
    for (w, frame) in zip(ws, frames):
        retval.append(w * frame.astype(np.int))
    return np.sum(retval, axis=0).astype(np.uint8)

def cv2imshow(ax, img):
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

fig, ax = plt.subplots()

cv2imshow(ax, mix([0.1, 0.2, 0.7], [frames[0], frames[9], frames[19]]))